# Import Packages

In [109]:
import os, sys, glob
import numpy as np
import SimpleITK as sitk
import sys
import os
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.utils import shuffle
%matplotlib inline
import pandas as pd
import numpy as np
from skimage.util.dtype import dtype_range
from skimage.util import img_as_ubyte
from skimage import exposure
from skimage.morphology import disk
from skimage.filters import rank
from scipy.ndimage import gaussian_filter
from skimage import data
from skimage import img_as_float
from skimage.morphology import reconstruction
from scipy import ndimage
import random
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import tensorflow as tf

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.layers import Lambda, RepeatVector, Reshape
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPool2D, UpSampling2D
from tensorflow.keras.layers import concatenate, add
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.utils import np_utils
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# Model Definitions

In [110]:
# Build U-Net model
dropout=0.2
hn = 'he_normal'

IMG_HEIGHT=240
IMG_WIDTH=240
IMG_CHANNELS=4


def unet(input_size = (IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)):
    
    inputs = Input(input_size)
    
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(conv4)
    drop4 = Dropout(dropout)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(conv5)
    drop5 = Dropout(dropout)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = hn)(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = hn)(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = hn)(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = hn)(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = hn)(conv9)
    #conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    
    conv10 = Conv2D(4, (1,1), activation = 'softmax')(conv9)
    
    model = Model(inputs = inputs, outputs = conv10)

    return model 



In [128]:
model = unet(input_size = (IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS))
model.compile(optimizer = Adam(lr = 0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [129]:
earlystopper = EarlyStopping(patience=8, verbose=1)
checkpointer = ModelCheckpoint(filepath = 'all_models/model_unet_oct_13_2023.h5',
                               verbose=1,
                               save_best_only=True, save_weights_only = False)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001, verbose=1,  cooldown=1)

# Load training data and make batch size

In [130]:
df_train= pd.read_csv('df_train.csv')
df_test= pd.read_csv('df_test.csv')
DATA = 'training_data/'
NUMPY_DIR = 'numpy_images/'

In [131]:
def get_img_batch(row, np_dir=NUMPY_DIR):
    
    im,lb = get_numpy_img_lbl(row['id'], np_dir)
    
    n_im = row['rmax']-row['rmin']
    rmin=row['rmin']
    rmax=row['rmax']
    
    return normalize_3D_image(im[rmin:rmax]), to_categorical(lb[rmin:rmax],4)

In [132]:
def get_df_img_batch(df_batch, np_dir=NUMPY_DIR):
    
        n_images = (df_batch.rmax - df_batch.rmin).sum()
        b_images = np.zeros((n_images, 240, 240, 4), np.float32)
        b_label = np.zeros((n_images, 240, 240, 4), np.int8)    
        ind=0
        for index, row in df_batch.iterrows():
 
            b_im, b_lb = get_img_batch(row, np_dir)
            n_im = b_im.shape[0]
            b_images[ind:ind+n_im] = b_im
            b_label[ind:ind+n_im] = b_lb
            ind+=n_im
               
        return b_images, b_label

In [133]:
def get_numpy_img_lbl(img_id = 'BraTS', np_dir=NUMPY_DIR):
    img=np.load(os.path.join(np_dir, img_id+'.npy'))
    lbl=np.load(os.path.join(np_dir, img_id+'_lbl.npy'))
    return img,lbl

In [134]:
def get_img_for_label(lab=2, axis=0, df=df_train,np_dir = NUMPY_DIR):
    
    img_id= random.choice(df[df['lab'+str(lab)] > 0].id.values)
    
    img,lbl = get_numpy_img_lbl(img_id, np_dir)
    ind = np.where(lbl==lab)
    k = random.randrange(len(ind[0]))
    
    if (axis==0):        
        return img[ind[0][k],:,:] , lbl[ind[0][k],:,:]
        
    lb = np.zeros((240,240),dtype=np.int)
    im = np.zeros((240,240,4),dtype=np.float32)
    
    if (axis==1):
        im[40:40+155,:,:]=img[:, ind[1][k],:,:]
        lb[40:40+155,:]=lbl[:, ind[1][k],:]
        return im,lb
    
    if (axis == 2):
        im[40:40+155,:,:]=img[:, :, ind[2][k],:]
        lb[40:40+155,:]=lbl[:,:,ind[2][k]]
        return im,lb
    return None

In [135]:
def normalize_3D_image(img):
    for z in range(img.shape[0]):
        for k in range(4):
            if (img[z,:,:,k].max()>0):
                img[z,:,:,k] /= img[z,:,:,k].max()
    return img

In [136]:
def normalize_2D_image(img):

        for c in range(4):
            if (img[:,:,c].max()>0):
                img[:,:,c] = img[:,:,c]/img[:,:,c].max()
        return img

In [137]:
def generate_faste_train_batch(batch_size = 12, df = df_train ,np_dir=NUMPY_DIR):
    
    batch_images = np.zeros((batch_size, 240, 240, 4), np.float32)
    batch_label = np.zeros((batch_size, 240, 240, 4), np.int8)    
    
    

    while 1:
        
        df_batch = df.sample(3)
        b_images, b_label = get_df_img_batch(df_batch, np_dir)                    
        b_images, b_label = shuffle(b_images, b_label)
        batch_images[0:batch_size//2]=b_images[0:batch_size//2]
        batch_label[0:batch_size//2]=b_label[0:batch_size//2]
        
        i=batch_size//2
        # lab 1
        nim = batch_size//4
        for j in range(nim):
            im,lbl = get_img_for_label(lab=1, axis=random.choice([0,1,2]), df=df)
            batch_images[i] = normalize_2D_image(im)
            batch_label[i] = to_categorical(lbl, 4)
            i+=1
                        
        # lab 3
        nim = batch_size//4
        for j in range(nim):
            im,lbl = get_img_for_label(lab=3, axis=random.choice([0,1,2]), df=df)
            batch_images[i] = normalize_2D_image(im)
            batch_label[i] = to_categorical(lbl, 4)
            i+=1

        batch_images, batch_label = shuffle(batch_images, batch_label)
            
        yield batch_images, batch_label

In [138]:
def generate_im_test_batch(n_images = 3, batch_size=3, df = df_test, np_dir=NUMPY_DIR):

    while 1:
         
        df_batch = df.sample(n_images)
        b_images, b_label = get_df_img_batch(df_batch, np_dir)                    
        b_images, b_label = shuffle(b_images, b_label)
        if (batch_size > 0):
            b_images = b_images[0:batch_size]
            b_label = b_label[0:batch_size]
            
        yield b_images, b_label

In [139]:
gen_train_fast = generate_faste_train_batch(batch_size=3)
bimg,blbl = next(gen_train_fast)
bimg.shape, blbl.shape

((3, 240, 240, 4), (3, 240, 240, 4))

In [140]:
gen_test_im = generate_im_test_batch(5)
imtest,lbtest = next(gen_test_im)
imtest.shape, lbtest.shape

((3, 240, 240, 4), (3, 240, 240, 4))

# Model Training start

In [ ]:
history = model.fit(gen_train_fast,
                                        validation_data = gen_test_im, validation_steps=1,
                                              steps_per_epoch=30,
                              epochs=100,
                    callbacks=[earlystopper, checkpointer, reduce_lr])

Epoch 1/100
30/30 [==============================] - 137s 5s/step - loss: 0.5909 - accuracy: 0.8695 - val_loss: 0.3538 - val_accuracy: 0.9767

Epoch 00001: val_loss improved from inf to 0.35379, saving model to all_models/model_unet_oct_13_2023.h5
Epoch 2/100
30/30 [==============================] - 55s 2s/step - loss: 0.0981 - accuracy: 0.9788 - val_loss: 0.1374 - val_accuracy: 0.9679

Epoch 00002: val_loss improved from 0.35379 to 0.13744, saving model to all_models/model_unet_oct_13_2023.h5
Epoch 3/100
30/30 [==============================] - 48s 2s/step - loss: 0.1076 - accuracy: 0.9699 - val_loss: 0.0987 - val_accuracy: 0.9775

Epoch 00003: val_loss improved from 0.13744 to 0.09871, saving model to all_models/model_unet_oct_13_2023.h5
Epoch 4/100
30/30 [==============================] - 67s 2s/step - loss: 0.0631 - accuracy: 0.9787 - val_loss: 0.0643 - val_accuracy: 0.9807

Epoch 00004: val_loss improved from 0.09871 to 0.06430, saving model to all_models/model_unet_oct_13_2023.h5

# Data preprocessing for testing

In [48]:
DATA = 'testing_data/'

NUMPY_DIR = 'numpy_images_test/'
NUMPY_DIR_LABEL = 'numpy_images_test_label/'

In [49]:
def read_img_sitk(img):
    inputImage = sitk.ReadImage( img )
    inputImage = sitk.Cast( inputImage, sitk.sitkFloat32 )
    image = sitk.GetArrayFromImage(inputImage)
    return image

In [50]:
def bbox2_3D(img):

    r = np.any(img, axis=(1, 2))
    c = np.any(img, axis=(0, 2))
    z = np.any(img, axis=(0, 1))

    rmin, rmax = np.where(r)[0][[0, -1]]
    cmin, cmax = np.where(c)[0][[0, -1]]
    zmin, zmax = np.where(z)[0][[0, -1]]

    return [rmin, rmax, cmin, cmax, zmin, zmax]

In [51]:
def read_image_into_numpy(dirpath):
    
    img_id = os.path.basename(dirpath)
    np_image=np.zeros((4, 155, 240, 240), dtype=np.float32)
    
    ## Flair
    flair_img = os.path.join(dirpath, img_id+'-t2f.nii.gz')
    if (not os.path.isfile(flair_img)):
        print(flair_img,' not found aborting')
        return None
    np_image[0] = read_img_sitk(flair_img)
        
    ## T1
    t1_img = os.path.join(dirpath, img_id+'-t1n.nii.gz')
    if (not os.path.isfile(t1_img)):
        print(t1_img,' not found aborting')
        return None
    np_image[1] = read_img_sitk(t1_img)
        
            
    ## T1CE
    t1ce_img = os.path.join(dirpath, img_id+'-t1c.nii.gz')
    if (not os.path.isfile(t1ce_img)):
        print(t1ce_img,' not found aborting')
        return None
    np_image[2] = read_img_sitk(t1ce_img)
    
        
    ## T2
    t2_img = os.path.join(dirpath, img_id+'-t2w.nii.gz')
    if (not os.path.isfile(t2_img)):
        print(t2_img,' not found aborting')
        return None
    np_image[3] = read_img_sitk(t2_img)

    return np_image

In [52]:
def read_lable_into_numpy(dirpath):
    
    img_id = os.path.basename(dirpath)
    np_image=np.zeros((155, 240, 240), dtype=np.int)
    
    ## lable
    lable_img = os.path.join(dirpath, img_id+'-seg.nii.gz')
    if (not os.path.isfile(lable_img)):
        print(lable_img,' not found aborting')
        return None
    np_image = read_img_sitk(lable_img).astype(int)

    return np_image

In [55]:
hgg_paths = []
for dirpath, dirnames, files in os.walk(DATA):
    if ('BraTS' in dirpath):
        hgg_paths.append(dirpath)

print(len(hgg_paths))

250


In [58]:
label_type_shrt = ['background', 'necrotic',
             'edema', 'enhancing']
label_type = ['background', 'necrotic and non-enhancing tumor', 'edema', 'enhancing tumor']

In [59]:
df = pd.DataFrame(columns=['id','lab0','lab1','lab2','lab3',
                           'rmin','rmax','cmin','cmax','zmin','zmax'])

df_val = pd.DataFrame(columns=['id','lab0','lab1','lab2','lab3'])

In [34]:
def fill_df_from_path(df, paths = hgg_paths):
    
    for f in paths:
        np_img = read_image_into_numpy(f)
        np_lbl = read_lable_into_numpy(f)
        
        
        new_img = np.zeros((155, 240, 240,4))
        for i in range(4):
            new_img[:,:,:,i] = np_img[i, :,:,:] 
            
        nimg = os.path.join(NUMPY_DIR,  os.path.basename(f)+'.npy')
        np.save(nimg, new_img)
        nlbl = os.path.join(NUMPY_DIR_LABEL,  os.path.basename(f)+'_lbl.npy')
        np.save(nlbl, np_lbl)

        lbls, repeats = np.unique(np_lbl, return_counts=True)
        lbl_counts=[0,0,0,0]
        for i in range(len(repeats)):
            lbl_counts[lbls[i]] = repeats[i]
        

        vals = [os.path.basename(f)] + lbl_counts + bbox2_3D(np_lbl)
        
        df.loc[len(df)] = vals
        
    return df

In [61]:
val_paths = []
for dirpath, dirnames, files in os.walk('testing_data/'):
    if ('BraTS' in dirpath):
        val_paths.append(dirpath)



def fill_df_from_path(df=df_val, paths = val_paths):
    
    for f in paths:
        np_img = read_image_into_numpy(f)

        new_img = np.zeros((155, 240, 240,4))
        for i in range(4):
            new_img[:,:,:,i] = np_img[i, :,:,:] 

        nimg = os.path.join('numpy_images_test',  os.path.basename(f)+'.npy')
        np.save(nimg, new_img)
        
        vals = [os.path.basename(f)]+[0,0,0,0]
        df.loc[len(df)] = vals
        
    return df

In [62]:
df_val = fill_df_from_path(df_val, paths = val_paths)

df_val.to_csv('df_validation.csv', index=False)

In [ ]:
df = fill_df_from_path(df, paths = hgg_paths)

In [ ]:
print(df.shape)

df.to_csv('df_validation.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2)

In [ ]:
df_train.to_csv('df_train.csv', index=False)
df_test.to_csv('df_test.csv',index=False)

# Prediction with trained model

In [142]:
pretrianed_brain_tumor_model=None

#pretrianed_brain_tumor_model = keras.models.load_model('./all_models/model_unet_for_brain_tumor.hdf5')
pretrianed_brain_tumor_model = load_model('all_models/model_unet_oct_13_2023.h5')



In [144]:
def get_pred(img, threshold=0.5):
    out_img=img.copy()
    out_img=np.where(out_img>threshold, 1,0)
    return out_img


def prediction_from_probabily_3D(img):
    
    int_image = get_pred(img)
    return lbl_from_cat(int_image)


def get_prediction_for_batch(pred_batch, threshold=0.5):
    
    out_batch = np.zeros((pred_batch.shape[0], 240, 240),dtype=np.int)
    
    for j in range(pred_batch.shape[0]):
        pred = get_prediction(pred_batch[j])
        if (pred.sum()>0):
            print(j, np.unique(pred , return_counts=True))
        out_batch[j] = lbl_from_cat(get_prediction(pred_batch[j]))
    return out_batch  


def get_label_from_pred_batch(labels_batch):
    
    batch = np.zeros((labels_batch.shape[0], 240, 240), np.uint8)
     
    for j in range(labels_batch.shape[0]):
        batch[j]=get_pred(labels_batch[j,:,:,0])+\
                get_pred(labels_batch[j,:,:,1])*2+\
        get_pred(labels_batch[j,:,:,2])*4

    return batch


def predict_3D_img_prob(np_file):
    
    np_img = np.load(np_file)
    for_pred_img = np.zeros((155, 240, 240, 4), np.float32)

    # Normalize image
    for_pred_img = normalize_3D_image(np_img)

    mdl_pred_img =  pretrianed_brain_tumor_model.predict(for_pred_img)

    #pred_label = prediction_from_probabily_3D(mdl_pred_img)

    return mdl_pred_img


def lbl_from_cat(cat_lbl):
    
    lbl=0
    if (len(cat_lbl.shape)==3):
        for i in range(1,4):
            lbl = lbl + cat_lbl[:,:,i]*i
    elif (len(cat_lbl.shape)==4):
        for i in range(1,4):
            lbl = lbl + cat_lbl[:,:,:,i]*i
    else:
        print('Error in lbl_from_cat', cat_lbl.shape)
        return None
    return lbl

In [145]:
VALIDATION_PRED_NUMPY_DIR = 'prediction_numpy/'
VALIDATION_PRED_NII_DIR = 'prediction_nii_fcnn/'
VALIDATION_NUMPY_DIR = 'numpy_images_test/'


for index, row in df_val.iterrows():

    img_id = row['id']

    nimg = os.path.join(VALIDATION_NUMPY_DIR, img_id+'.npy')
    pred_stats = predict_3D_img_prob(nimg)

    pred = prediction_from_probabily_3D(pred_stats)

    out_img = os.path.join(VALIDATION_PRED_NUMPY_DIR, img_id+'_pred.npy')
    np.save(out_img, pred)
    
    pred = np.where(pred==3,4, pred)
    out_nii = os.path.join(VALIDATION_PRED_NII_DIR, img_id+'.nii.gz')

    sitk_img = sitk.GetImageFromArray(pred)
    sitk.WriteImage(sitk_img , out_nii)